In [3]:
import netCDF4 as nc
import numpy as np
import numpy.ma as ma
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import cdsapi
import os
import cartopy.feature as cfeature
from cartopy.io.shapereader import natural_earth, Reader
from shapely.geometry import Point
from shapely.prepared import prep

## Informações

**ERA5** <br>
**link**: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=overview <br>
**Dataset**: reanalysis-era5-single-levels-monthly-means <br>
**Product type**: monthly_averaged_reanalysis <br>
**Resolution**: 0.25 <br>
**Variables:** <br>
mean_eastward_turbulent_surface_stress (N m**-2) <br>
mean_northward_turbulent_surface_stress (N m**-2) <br>
mean_surface_downward_short_wave_radiation_flux (W m**-2) <br>
mean_surface_latent_heat_flux (W m**-2) <br>
mean_surface_net_long_wave_radiation_flux (W m**-2) <br>
mean_surface_sensible_heat_flux (W m**-2) <br>
mean_surface_net_short_wave_radiation_flux (W m**-2) <br>
10m_u_component_of_wind (m s**-1) <br>
10m_v_component_of_wind (m s**-1) <br>
10m_wind_speed (m s**-1) <br>
2m_temperature (K) <br>
total_precipitation (m) <br>
evaporation (m) <br>
surface_pressure (Pa) <br>
sea_surface_temperature (ºC)

<br>

**ERA5** <br>
**link**: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-monthly-means?tab=overview <br>
**Dataset**: reanalysis-era5-pressure-levels-monthly-means <br>
**Product type**: monthly_averaged_reanalysis <br>
**Resolution**: 0.25 <br>
**Variables:** <br>
relative_humidity (%) <br>
<br>

**WOA18** <br>
**link**: https://www.ncei.noaa.gov/data/oceans/woa/WOA18/DATA/ (baixar manualmente os arquivos do 1 ao 12) <br>
**Documentation**: https://www.ncei.noaa.gov/sites/default/files/2022-06/woa18documentation.pdf <br>
**Product type**: decav (avarage of six decadal means from 1955 to 2017) <br>
**Resolution**: 0.25 <br>
**Variables:** <br>
sea surface salinity (PSU)

<br>

**COADS05**
**link**: https://iridl.ldeo.columbia.edu/SOURCES/.DASILVA/.SMD94/.climatology/ <br>
**Variables:** <br>
zonal wind stress (taux) (N m**-2) <br>
meridional wind stress (tauy) (N m**-2) <br>
constrained net outgoing (netheat) (w m**-2) <br>
incoming short wave radiation (shortrad) (w m**-2) <br>
wind speed (w3) m s**-1 <br>
sea level air temperature (sat) (ºC) <br>
emp (evaporation rate - precipitation rate) (mm 3hour**-1) <br>
precip (mm 3hour**-1) <br>
sea level air density (airdens) (kg m**-3) <br>
sea level specific humidity (qsea) (g kg**-1) <br>
sea surface temperature (sst) (ºC) <br>
sea surface salinity (sss) (PSU) <br>

## Cálculos

### constrained net outgoing (netheat)

netheat = mean_surface_net_short_wave_radiation_flux + mean_surface_latent_heat_flux + mean_surface_sensible_heat_flux + mean_surface_net_long_wave_radiation_flux <br>
Fonte: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019EA000988

### emp (evaporation rate - precipitation rate)

total_evaporation (m) > total_evaporation (mm 3hour**-1): <br>
(total_evaporation * 1000)/8

evaporation (m) > evaporation (mm 3hour**-1): <br>
(evaporation * 1000)/8

### sea level specific humidity

**Constantes** <br>
rd = 287.0597 (J kg**-1 K**-1) <br>
rv = 461.5250 (J kg**-1 K**-1) <br>
t0 = 273.16 (K) <br>
a1 = 611.21 (Pa) <br>
a3 = 17.502 (K) <br>
a4 = 32.19 (K) <br>

**Calcular es(T) usando numpy.exp** <br>
es_T = a1 * np.exp(a3 * ((sea_level_air_temperature-t0)/(sea_level_air_temperature-a4))) <br>
qsea = ((rd/rv) * es_T)/(surface_pressure-(1-(rd/rv)) * es_T) > kg kg**-1 <br>

**Converter specific humidity de kg kg-1 para g kg-1** <br>
qsea = qsea * 1000

**Fonte**: <br>
https://www.ecmwf.int/en/elibrary/81370-ifs-documentation-cy48r1-part-iv-physical-processes <br>
cap IV <br>
eq. 7.4 and 7.5 <br>
https://forum.ecmwf.int/t/how-to-calculate-hus-at-2m-huss/1254

### sea level air density

**Fórmula (DASILVA, 1994)**: <br>
airdens (kg/m³) = p/R*sat_K*(1+0.61*qs), <br><br>
onde: <br>
p = sea level pressure (mb ou Pa) <br>
R = constante do gás para ar seco: 287.058 (J/kg*K)<br>
sat_K = temperatura do ar superficial (K) <br>
qs = umidade específica (kg/kg) <br> <br>

fonte: A. da Silva, A. C. Young, S. Levitus. Atlas of Surface Marine Data 1994, Volume 1: Algorithms and Procedures, number 6, 1994

## Dataset download
Seguir as instruções para cumprir os requisitos necessários para baixar arquivos do ERA5 via Python: <br>
1. acesse https://cds.climate.copernicus.eu/ e crie um loguin
2. vá para o diretório C:Users\username
3. crie um arquivo de texto com o nome .cdsapirc
4. altere a extensão do arquivo para .CDSAPIRC
5. acesse https://cds.climate.copernicus.eu/user/ e anote o API Key e UID
6. abra o arquivo .cdsapirc em um editor de texto
7. cole as seguintes informações dentro do arquivo:
- url: https://cds.climate.copernicus.eu/api/v2
- key: xxxxx(inserir aqui o UID):xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx(inserir aqui o API Key) <br>
8. instalar a biblioteca cdsapi (pip install cdsapi)
<br> <br>
Fonte: https://stackoverflow.com/questions/66288678/cds-toolbox-exception-missing-incomplete-configuration <br> <br>

Seguir as instruções para cumprir os requisitos necessários para baixar arquivos do MarineCopernicus via Python: <br>
1. acesse https://data.marine.copernicus.eu/products, crie uma conta e faça o loguin
2. instalar a biblioteca copernicusmarine (pip install copernicusmarine)

### reanalysis-era5-single-levels-monthly-means

In [4]:
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'netcdf',
        'product_type': 'monthly_averaged_reanalysis',
        'variable': [
           'mean_eastward_turbulent_surface_stress', 'mean_northward_turbulent_surface_stress', 'mean_surface_downward_short_wave_radiation_flux', 'mean_surface_latent_heat_flux',
            'mean_surface_net_long_wave_radiation_flux', 'mean_surface_net_short_wave_radiation_flux', 'mean_surface_sensible_heat_flux', '10m_wind_speed', '2m_temperature', 'total_precipitation', 'evaporation', 'surface_pressure', 
            '10m_u_component_of_wind', '10m_v_component_of_wind','sea_surface_temperature',
        ],
        'year': [
            '2000','2001','2002',
            '2003','2004','2005',
            '2006','2007','2008',
            '2009','2010', '2011', '2012',
            '2013', '2014', '2015',
            '2016', '2017', '2018',
            '2016', '2020', '2021',
            '2022', '2023',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
        'area': [
            90, -180, -90,
            180,
        ],
    },
    'reanalysis_era5_single_levels_monthly_means.nc')

2024-07-29 14:14:11,236 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-07-29 14:14:11,248 WARNING MOVE TO CDS-Beta
2024-07-29 14:14:11,248 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-07-29 14:14:11,797 INFO Request is completed
2024-07-29 14:14:11,801 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data6/adaptor.mars.internal-1722255120.2278109-22892-1-8f536ac4-4a9b-4f6e-b9f6-9581f0df46b0.nc to reanalysis_era5_single_levels_monthly_means.nc (8G)
202

Result(content_length=8596641732,content_type=application/x-netcdf,location=https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data6/adaptor.mars.internal-1722255120.2278109-22892-1-8f536ac4-4a9b-4f6e-b9f6-9581f0df46b0.nc)

In [3]:
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels-monthly-means',
    {
        'format': 'netcdf',
        'product_type': 'monthly_averaged_reanalysis',
        'variable': 'relative_humidity',
        'pressure_level': '1000',
        'year': [
            '2000','2001','2002',
            '2003','2004','2005',
            '2006','2007','2008',
            '2009','2010', '2011', '2012',
            '2013', '2014', '2015',
            '2016', '2017', '2018',
            '2016', '2020', '2021',
            '2022', '2023',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
        'area': [
            90, -180, -90,
            180,
        ],
    },
    'reanalysis_era5_pressure_levels_monthly_means.nc')

2024-07-09 17:44:06,128 INFO Welcome to the CDS
2024-07-09 17:44:06,128 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels-monthly-means
2024-07-09 17:44:06,508 INFO Request is queued
2024-07-09 17:44:07,671 INFO Request is running
2024-07-09 17:46:59,132 INFO Request is completed
2024-07-09 17:46:59,132 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data0/adaptor.mars.internal-1720547156.363983-24321-1-ef4a61d7-a8dd-4a71-ab86-f1e45075d51e.nc to rhumidity_data.nc (546.6M)
2024-07-09 17:49:43,598 INFO Download rate 3.3M/s    


Result(content_length=573119344,content_type=application/x-netcdf,location=https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data0/adaptor.mars.internal-1720547156.363983-24321-1-ef4a61d7-a8dd-4a71-ab86-f1e45075d51e.nc)

## Tratamento

In [5]:
dataset = nc.Dataset('reanalysis_era5_single_levels_monthly_means.nc')
dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2024-07-29 12:14:16 GMT by grib_to_netcdf-2.28.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf -S param -o /cache/data6/adaptor.mars.internal-1722255120.2278109-22892-1-8f536ac4-4a9b-4f6e-b9f6-9581f0df46b0.nc /cache/tmp/8f536ac4-4a9b-4f6e-b9f6-9581f0df46b0-adaptor.mars.internal-1722254620.613707-22892-1-tmp.grib
    dimensions(sizes): longitude(1440), latitude(721), time(276)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 metss(time, latitude, longitude), int16 mntss(time, latitude, longitude), int16 msdwswrf(time, latitude, longitude), int16 mslhf(time, latitude, longitude), int16 msnlwrf(time, latitude, longitude), int16 msnswrf(time, latitude, longitude), int16 msshf(time, latitude, longitude), int16 si10(time, latitude, longitude), int16 t2m(time, latitude, longitude), int16 tp(time, lat

In [7]:
lat = np.array(dataset['latitude'][:],dtype=np.float64)
lat = np.flip(lat)
lon = np.array(dataset['longitude'][:],dtype=np.float64)
time_num = np.array(dataset['time'][:])

### Criando a máscara no continente

In [9]:
# Criar arrays de grid
lon_grid, lat_grid = np.meshgrid(lon, lat)

# Baixar os dados de limites de terra
land_shp_fname = natural_earth(resolution='110m', category='physical', name='land')
land_geom = list(Reader(land_shp_fname).geometries())
land_geom_prep = [prep(geom) for geom in land_geom]

# Função para verificar se um ponto está na terra
def is_land(lon, lat):
    point = Point(lon, lat)
    return any(geom.contains(point) for geom in land_geom_prep)

# Criar a máscara
mask = np.zeros(lon_grid.shape, dtype=bool)

# Iterar sobre o grid e preencher a máscara
for i in range(lat_grid.shape[0]):
    for j in range(lon_grid.shape[1]):
        mask[i, j] = is_land(lon_grid[i, j], lat_grid[i, j])
        
mask = np.expand_dims(mask, axis=0)
mask = np.tile(mask, (12, 1, 1))

### Wind stress (N m**-2)

In [13]:
# extraindo dados essenciais para criar novas variáveis da climatologia
mes_clim = np.arange(0,12)

sum_meses = np.zeros(12)

In [14]:
# criando novas variáveis para armazenar a climatologia
taux = np.zeros((len(mes_clim), len(lat), len(lon))) # mean_eastward_turbulent_surface_stress
tauy = np.zeros((len(mes_clim), len(lat), len(lon))) # mean_northward_turbulent_surface_stress

metss = np.array(dataset['metss'][:,:,:], dtype=np.float64)
metss = np.flip(metss, axis=1)
mntss = np.array(dataset['mntss'][:,:,:], dtype=np.float64)
mntss = np.flip(mntss, axis=1)


# criando data inicial 1950-01-01
start_datetime = datetime(1950,1,1,0,0,0)
    
dt_real=[]
    
#convertendo time_num (float) para data
for t in time_num: 
        dt_real.append(start_datetime + timedelta(hours = int(t)))

for i in range(0,len(time_num)): 
    mes_data=int(dt_real[i].strftime("%m"))
    # loop para todos os meses da climatologia
    for j in mes_clim:
         # verificando se o mes do dado corresponde a um mes da climatologia e salvando 
        if mes_data == j+1:
            # fazendo a soma dos valores e armazenando em cada posiçao específica 
                taux[j,:,:] = taux[j,:,:] + metss[i,:,:]
                tauy[j,:,:] = tauy[j,:,:] + mntss[i,:,:]
                # atualizando o contador
                sum_meses[j] = sum_meses[j] + 1;
                
                print('Variável sum_meses: ',sum_meses)
                
#Iniciando segundo loop - obtendo os valores médios
for i in mes_clim:
    taux[i,:,:] = np.divide(taux[i,:,:], sum_meses[i]) # N m**-2
    tauy[i,:,:] = np.divide(tauy[i,:,:], sum_meses[i]) # N m**-2
    
taux = ma.masked_array(taux,mask=mask,fill_value=-1.0e10)
tauy = ma.masked_array(tauy,mask=mask,fill_value=-1.0e10)

Variável sum_meses:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2

### Netheat (W m**-2)

In [85]:
# extraindo dados essenciais para criar novas variáveis da climatologia
mes_clim = np.arange(0,12)

sum_meses = np.zeros(12)

In [87]:
# criando novas variáveis para armazenar a climatologia
clim_msdwswrf = np.zeros((len(mes_clim), len(lat), len(lon))) # mean_surface_downward_short_wave_radiation_flux
clim_mslhf = np.zeros((len(mes_clim), len(lat), len(lon))) # mean_surface_latent_heat_flux
clim_msnlwrf = np.zeros((len(mes_clim), len(lat), len(lon))) # mean_surface_net_long_wave_radiation_flux
clim_msshf = np.zeros((len(mes_clim), len(lat), len(lon))) # mean_surface_sensible_heat_flux
clim_msnswrf = np.zeros((len(mes_clim), len(lat), len(lon))) # mean_surface_net_long_wave_radiation_flux

msdwswrf = np.array(dataset['msdwswrf'][:,:,:],dtype=np.float64)
msdwswrf = np.flip(msdwswrf, axis=1)
mslhf = np.array(dataset['mslhf'][:,:,:],dtype=np.float64) 
mslhf = np.flip(mslhf, axis=1)
msnlwrf = np.array(dataset['msnlwrf'][:,:,:],dtype=np.float64)
msnlwrf = np.flip(msnlwrf, axis=1)
msshf = np.array(dataset['msshf'][:,:,:],dtype=np.float64) 
msshf = np.flip(msshf, axis=1)
msnswrf = np.array(dataset['msnswrf'][:,:,:],dtype=np.float64) 
msnswrf = np.flip(msnswrf, axis=1)

# criando data inicial 1950-01-01
start_datetime = datetime(1950,1,1,0,0,0)
    
dt_real=[]
    
#convertendo time_num (float) para data
for t in time_num: 
        dt_real.append(start_datetime + timedelta(hours = int(t)))

for i in range(0,len(time_num)): 
    mes_data=int(dt_real[i].strftime("%m"))
    # loop para todos os meses da climatologia
    for j in mes_clim:
         # verificando se o mes do dado corresponde a um mes da climatologia e salvando 
        if mes_data == j+1:
            # fazendo a soma dos valores e armazenando em cada posiçao específica 
                clim_msdwswrf[j,:,:] = clim_msdwswrf[j,:,:] + msdwswrf[i,:,:]
                clim_mslhf[j,:,:] = clim_mslhf[j,:,:] + mslhf[i,:,:]
                clim_msnlwrf[j,:,:] = clim_msnlwrf[j,:,:] + msnlwrf[i,:,:]
                clim_msshf[j,:,:] = clim_msshf[j,:,:] + msshf[i,:,:]
                clim_msnswrf[j,:,:] = clim_msnswrf[j,:,:] + msnswrf[i,:,:]
                # atualizando o contador
                sum_meses[j] = sum_meses[j] + 1;
                
                print('Variável sum_meses: ',sum_meses)
                
#Iniciando segundo loop - obtendo os valores médios
for i in mes_clim:
    clim_msdwswrf[i,:,:] = np.divide(clim_msdwswrf[i,:,:], sum_meses[i])
    clim_mslhf[i,:,:] = np.divide(clim_mslhf[i,:,:], sum_meses[i])
    clim_msnlwrf[i,:,:] = np.divide(clim_msnlwrf[i,:,:], sum_meses[i])
    clim_msshf[i,:,:] = np.divide(clim_msshf[i,:,:], sum_meses[i])
    clim_msnswrf[i,:,:] = np.divide(clim_msnswrf[i,:,:], sum_meses[i])
    
netheat = clim_msnswrf + clim_mslhf + clim_msnlwrf + clim_msshf # W m**-2
shortrad = clim_msdwswrf

netheat = ma.masked_array(netheat,mask=mask,fill_value=-1.0e10)
shortrad = ma.masked_array(shortrad,mask=mask,fill_value=-1.0e10)

Variável sum_meses:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2

### Wind speed (m s**-1)

In [29]:
# extraindo dados essenciais para criar novas variáveis da climatologia
mes_clim = np.arange(0,12)

sum_meses = np.zeros(12)

In [30]:
# criando novas variáveis para armazenar a climatologia
u3 = np.zeros((len(mes_clim), len(lat), len(lon))) # 10m_u_component_of_wind
v3 = np.zeros((len(mes_clim), len(lat), len(lon))) # 10m_v_component_of_wind
w3 = np.zeros((len(mes_clim), len(lat), len(lon))) # 10m_wind_speed 

u10 = np.array(dataset['u10'][:,:,:], dtype=np.float64)
u10 = np.flip(u10, axis=1)
v10 = np.array(dataset['v10'][:,:,:], dtype=np.float64)
v10 = np.flip(v10, axis=1)
si10 = np.array(dataset['si10'][:,:,:], dtype=np.float64)
si10 = np.flip(si10, axis=1)

# criando data inicial 1950-01-01
start_datetime = datetime(1950,1,1,0,0,0)
    
dt_real=[]
    
#convertendo time_num (float) para data
for t in time_num: 
        dt_real.append(start_datetime + timedelta(hours = int(t)))

for i in range(0,len(time_num)): 
    mes_data=int(dt_real[i].strftime("%m"))
    # loop para todos os meses da climatologia
    for j in mes_clim:
         # verificando se o mes do dado corresponde a um mes da climatologia e salvando 
        if mes_data == j+1:
            # fazendo a soma dos valores e armazenando em cada posiçao específica 
                u3[j,:,:] = u3[j,:,:] + u10[i,:,:]
                v3[j,:,:] = v3[j,:,:] + v10[i,:,:]
                w3[j,:,:] = w3[j,:,:] + si10[i,:,:]
                # atualizando o contador
                sum_meses[j] = sum_meses[j] + 1;
                
                print('Variável sum_meses: ',sum_meses)
                
#Iniciando segundo loop - obtendo os valores médios
for i in mes_clim:
    u3[i,:,:] = np.divide(u3[i,:,:], sum_meses[i]) # m s**-1
    v3[i,:,:] = np.divide(v3[i,:,:], sum_meses[i]) # m s**-1
    w3[i,:,:] = np.divide(w3[i,:,:], sum_meses[i]) # m s**-1
    
u3 = ma.masked_array(u3,mask=mask,fill_value=-1.0e10)
v3 = ma.masked_array(v3,mask=mask,fill_value=-1.0e10)
w3 = ma.masked_array(w3,mask=mask,fill_value=-1.0e10)

Variável sum_meses:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2

### Sea level air temperature (ºC)

In [11]:
# extraindo dados essenciais para criar novas variáveis da climatologia
mes_clim = np.arange(0,12)

sum_meses = np.zeros(12)

In [13]:
# criando novas variáveis para armazenar a climatologia
sat = np.zeros((len(mes_clim), len(lat), len(lon))) # 2m_temperature

t2m = np.array(dataset['t2m'][:,:,:], dtype=np.float64)
t2m = np.flip(t2m, axis=1)

# criando data inicial 1950-01-01
start_datetime = datetime(1950,1,1,0,0,0)
    
dt_real=[]
    
#convertendo time_num (float) para data
for t in time_num: 
        dt_real.append(start_datetime + timedelta(hours = int(t)))

for i in range(0,len(time_num)): 
    mes_data=int(dt_real[i].strftime("%m"))
    # loop para todos os meses da climatologia
    for j in mes_clim:
         # verificando se o mes do dado corresponde a um mes da climatologia e salvando 
        if mes_data == j+1:
            # fazendo a soma dos valores e armazenando em cada posiçao específica 
                sat[j,:,:] = sat[j,:,:] + t2m[i,:,:]
                # atualizando o contador
                sum_meses[j] = sum_meses[j] + 1;
                
                print('Variável sum_meses: ',sum_meses)
                
#Iniciando segundo loop - obtendo os valores médios
for i in mes_clim:
    sat[i,:,:] = np.divide(sat[i,:,:], sum_meses[i]) # K

sat = sat - 273.15 # ºC

sat = ma.masked_array(sat,mask=mask,fill_value=-1.0e10)

Variável sum_meses:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2

### emp (mm 3hour**-1)

In [40]:
# extraindo dados essenciais para criar novas variáveis da climatologia
mes_clim = np.arange(0,12)

sum_meses = np.zeros(12)

In [42]:
# criando novas variáveis para armazenar a climatologia
precip = np.zeros((len(mes_clim), len(lat), len(lon))) # total_precipitation
evaporation = np.zeros((len(mes_clim), len(lat), len(lon))) # evaporation

tp = np.array(dataset['tp'][:,:,:], dtype=np.float64)
tp = np.flip(tp, axis=1)
e = np.array(dataset['e'][:,:,:], dtype=np.float64)
e = np.flip(e, axis=1)

# criando data inicial 1950-01-01
start_datetime = datetime(1950,1,1,0,0,0)
    
dt_real=[]
    
#convertendo time_num (float) para data
for t in time_num: 
        dt_real.append(start_datetime + timedelta(hours = int(t)))

for i in range(0,len(time_num)): 
    mes_data=int(dt_real[i].strftime("%m"))
    # loop para todos os meses da climatologia
    for j in mes_clim:
         # verificando se o mes do dado corresponde a um mes da climatologia e salvando 
        if mes_data == j+1:
            # fazendo a soma dos valores e armazenando em cada posiçao específica 
                precip[j,:,:] = precip[j,:,:] + tp[i,:,:]
                evaporation[j,:,:] = evaporation[j,:,:] + e[i,:,:]
                # atualizando o contador
                sum_meses[j] = sum_meses[j] + 1;
                
                print('Variável sum_meses: ',sum_meses)
                
#Iniciando segundo loop - obtendo os valores médios
for i in mes_clim:
    precip[i,:,:] = np.divide(precip[i,:,:], sum_meses[i]) # m
    evaporation[i,:,:] = np.divide(evaporation[i,:,:], sum_meses[i]) # m
    
evaporation = (evaporation*1000)/8 #mm/(3hour)
precip = (precip*1000)/8 #mm/(3hour)
emp = -evaporation - precip

emp = ma.masked_array(emp,mask=mask,fill_value=-1.0e10)
precip = ma.masked_array(precip,mask=mask,fill_value=-1.0e10)

Variável sum_meses:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2

### Surface pressure (Pa)

In [15]:
# extraindo dados essenciais para criar novas variáveis da climatologia
mes_clim = np.arange(0,12)

sum_meses = np.zeros(12)

In [17]:
# criando novas variáveis para armazenar a climatologia
surface_pressure = np.zeros((len(mes_clim), len(lat), len(lon))) # surface_pressure

sp = np.array(dataset['sp'][:,:,:], dtype=np.float64)
sp = np.flip(sp, axis=1)

# criando data inicial 1950-01-01
start_datetime = datetime(1950,1,1,0,0,0)
    
dt_real=[]
    
#convertendo time_num (float) para data
for t in time_num: 
        dt_real.append(start_datetime + timedelta(hours = int(t)))

for i in range(0,len(time_num)): 
    mes_data=int(dt_real[i].strftime("%m"))
    # loop para todos os meses da climatologia
    for j in mes_clim:
         # verificando se o mes do dado corresponde a um mes da climatologia e salvando 
        if mes_data == j+1:
            # fazendo a soma dos valores e armazenando em cada posiçao específica 
                surface_pressure[j,:,:] = surface_pressure[j,:,:] + sp[i,:,:]
                # atualizando o contador
                sum_meses[j] = sum_meses[j] + 1;
                
                print('Variável sum_meses: ',sum_meses)
                
#Iniciando segundo loop - obtendo os valores médios
for i in mes_clim:
    surface_pressure[i,:,:] = np.divide(surface_pressure[i,:,:], sum_meses[i]) # Pa
    
surface_pressure = ma.masked_array(surface_pressure,mask=mask,fill_value=-1.0e10)

Variável sum_meses:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2

### Sea level specific humidity (g kg**-1) 

**Constantes** <br>
rd = 287.0597 (J kg**-1 K**-1) <br>
rv = 461.5250 (J kg**-1 K**-1) <br>
t0 = 273.16 (K) <br>
a1 = 611.21 (Pa) <br>
a3 = 17.502 (K) <br>
a4 = 32.19 (K) <br>

utilizar sat (sea air temperature) em kelvin

**Calcular es(T) usando numpy.exp** <br>
es_T = a1 * np.exp(a3 * ((sea_level_air_temperature-t0)/(sea_level_air_temperature-a4))) <br>
qsea = ((rd/rv) * es_T)/(surface_pressure-(1-(rd/rv)) * es_T) > kg kg**-1 <br>

**Converter specific humidity de kg kg-1 para g kg-1** <br>
qsea = qsea * 1000

**Fonte**: <br>
https://www.ecmwf.int/en/elibrary/81370-ifs-documentation-cy48r1-part-iv-physical-processes <br>
cap IV <br>
eq. 7.4 and 7.5 <br>
https://forum.ecmwf.int/t/how-to-calculate-hus-at-2m-huss/1254

In [19]:
rd = 287.0597 # Constante específica do gás para o ar seco em J kg**-1 K**-1
rv = 461.5250 # Constante específica do gás para o vapor de água em J kg**-1 K**-1
t0 = 273.16 # K
a1 = 611.21 # Pa
a3 = 17.502 # K
a4 = 32.19 # K
sat_K = sat + 273.15 # K

es_T = a1*np.exp(a3*((sat_K-t0)/(sat_K-a4)))
qsea = ((rd/rv)*es_T)/(surface_pressure-(1-(rd/rv))*es_T) # kg kg**-1

qsea = qsea*1000 # g kg**-1

qsea = ma.masked_array(qsea,mask=mask,fill_value=-1.0e10)

### Sea level air density (kg m**-3)

**Fórmula (DASILVA, 1994)**: <br>
airdens (kg/m³) = p/R*sat_K*(1+0.61*qs), <br><br>
onde: <br>
p = sea level pressure (mb ou Pa) <br>
R = constante do gás para ar seco: 287.058 (J/kg*K)<br>
sat_K = temperatura do ar superficial (K) <br>
qs = umidade específica (kg/kg) <br> <br>

fonte: A. da Silva, A. C. Young, S. Levitus. Atlas of Surface Marine Data 1994, Volume 1: Algorithms and Procedures, number 6, 1994

In [21]:
R = 287.058 # J/kg*K
qs = qsea/1000 # kg/kg
airdens = surface_pressure/(R*sat_K*(1+0.61*qs)) # kg/m³

airdens = ma.masked_array(airdens,mask=mask,fill_value=-1.0e10)

### SST (ºC)

In [43]:
# extraindo dados essenciais para criar novas variáveis da climatologia
mes_clim = np.arange(0,12)

sum_meses = np.zeros(12)

In [44]:
# criando novas variáveis para armazenar a climatologia
sst = np.zeros((len(mes_clim), len(lat), len(lon))) # sea_surface_temperature

temp = np.array(dataset['sst'][:,:,:], dtype=np.float64)
temp = np.flip(temp, axis=1)

# criando data inicial 1950-01-01
start_datetime = datetime(1950,1,1,0,0,0)
    
dt_real=[]
    
#convertendo time_num (float) para data
for t in time_num: 
        dt_real.append(start_datetime + timedelta(hours = int(t)))

for i in range(0,len(time_num)): 
    mes_data=int(dt_real[i].strftime("%m"))
    # loop para todos os meses da climatologia
    for j in mes_clim:
         # verificando se o mes do dado corresponde a um mes da climatologia e salvando 
        if mes_data == j+1:
            # fazendo a soma dos valores e armazenando em cada posiçao específica 
                sst[j,:,:] = sst[j,:,:] + temp[i,:,:]
                # atualizando o contador
                sum_meses[j] = sum_meses[j] + 1;
                
                print('Variável sum_meses: ',sum_meses)
                
#Iniciando segundo loop - obtendo os valores médios
for i in mes_clim:
    sst[i,:,:] = np.divide(sst[i,:,:], sum_meses[i]) # K
    
sst = sst - 273.15 # ºC
sst = np.where(sst<=-1000,np.nan,sst) # retirar o fillvalue (que preenche os continentes)

sst = ma.masked_array(sst,mask=mask,fill_value=-1.0e10)

Variável sum_meses:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Variável sum_meses:  [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 2.]
Variável sum_meses:  [2. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2

In [45]:
# Iterar sobre cada 'time' para realizar a interpolação
for t in range(sst.shape[0]):
    # Identificar pontos válidos em C e pontos inválidos em E para o tempo t
    valid_points_C = ~np.isnan(sst[t])
    invalid_points_E = np.isnan(sst[t])

    # Coordenadas dos pontos válidos em C
    lat_valid, lon_valid = np.meshgrid(lat, lon, indexing='ij')
    points_C = np.array([lat_valid[valid_points_C], lon_valid[valid_points_C]]).T
    values_C = sst[t][valid_points_C]

    # Coordenadas dos pontos inválidos em E
    points_to_interpolate = np.array([lat_valid[invalid_points_E], lon_valid[invalid_points_E]]).T

    # Interpolação usando valores de C para substituir NaNs em E
    sst[t][invalid_points_E] = griddata(
        points_C,          # pontos válidos de C
        values_C,          # valores válidos de C
        points_to_interpolate,  # pontos inválidos de E para interpolar
        method='cubic'
    )

# E agora deve ter os valores NaN interpolados usando valores válidos de C

### SSS (PSU) 

In [46]:
# Dimensões: time(12), lat(720), lon(1440)
sss = np.zeros((12, 720, 1440))
mm = ['01','02','03','04','05','06','07','08','09','10','11','12']
for i in range(0,len(mm)):
    dataset_so = nc.Dataset('woa18_decav_s'+mm[i]+'_04.nc')
    s_an = dataset_so['s_an'][:,0,:,:]
    sss[i,:,:] = s_an[:,:,:]
    
sss = np.where(sss==9.969209968386869e+36, np.nan, sss)
lat_so = np.array(dataset_so['lat'][:],dtype=np.float64)
lon_so = np.array(dataset_so['lon'][:],dtype=np.float64)
print('antes da interpolação :', sss.shape)

lat_interp = np.interp(lat,lat_so,lat_so)
interp_func = interp1d(lat_so, sss, axis=1, fill_value="extrapolate")
sss = interp_func(lat)
print('depois da interpolação :', sss.shape)

sss = ma.masked_array(sss,mask=mask,fill_value=-1.0e10)

antes da interpolação : (12, 720, 1440)
depois da interpolação : (12, 721, 1440)


In [47]:
# Iterar sobre cada 'time' para realizar a interpolação
for t in range(sss.shape[0]):
    # Identificar pontos válidos em C e pontos inválidos em E para o tempo t
    valid_points_C = ~np.isnan(sss[t])
    invalid_points_E = np.isnan(sss[t])

    # Coordenadas dos pontos válidos em C
    lat_valid, lon_valid = np.meshgrid(lat, lon, indexing='ij')
    points_C = np.array([lat_valid[valid_points_C], lon_valid[valid_points_C]]).T
    values_C = sss[t][valid_points_C]

    # Coordenadas dos pontos inválidos em E
    points_to_interpolate = np.array([lat_valid[invalid_points_E], lon_valid[invalid_points_E]]).T

    # Interpolação usando valores de C para substituir NaNs em E
    sss[t][invalid_points_E] = griddata(
        points_C,          # pontos válidos de C
        values_C,          # valores válidos de C
        points_to_interpolate,  # pontos inválidos de E para interpolar
        method='cubic'
    )

# E agora deve ter os valores NaN interpolados usando valores válidos de C

## Convert netCDF4

In [46]:
time_nc = np.arange(1,13)
def get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value):
    """
    Cria um arquivo NetCDF com a variável fornecida e seus atributos.

    Parâmetros:
    - filename (str): Nome do arquivo NetCDF a ser criado.
    - timedim (int): Tamanho da dimensão temporal.
    - latdim (int): Tamanho da dimensão latitudinal.
    - londim (int): Tamanho da dimensão longitudinal.
    - lat (array): Dados de latitude.
    - lon (array): Dados de longitude.
    - time (array): Dados de tempo.
    - nome_variavel (str): Nome da variável a ser armazenada.
    - variable (array): Dados da variável a ser armazenada.
    - variable_units (str): Unidade da variável.
    - variable_history (str): Histórico da variável.
    - missing_value (float): Valor representando dados ausentes na variável.

    Retorna:
    - Mensagem de sucesso.
    """

    # Criando o arquivo NetCDF
    ncid = nc.Dataset(filename, 'w', format='NETCDF4')

    # Definindo as dimensões
    time_dim = ncid.createDimension('T', timedim)  # Dimensão temporal
    lat_dim = ncid.createDimension('Y', latdim)  # Dimensão latitudinal
    lon_dim = ncid.createDimension('X', londim)  # Dimensão longitudinal

    # Criando as variáveis associadas às dimensões
    T = ncid.createVariable('T', np.float64, ('T',))
    Y = ncid.createVariable('Y', np.float32, ('Y',))
    X = ncid.createVariable('X', np.float32, ('X',))

    T.long_name = 'Time'
    Y.long_name = 'latitude'
    X.long_name = 'longitude'

    # Atribuindo dados às variáveis de dimensão
    T[:] = time_nc
    Y[:] = lat
    X[:] = lon

    # Criando uma variável de dados com as dimensões definidas
    var = ncid.createVariable(var_str, np.float32, ('T', 'Y', 'X'),fill_value=missing_value)

    # Adicionando dados à variável, incluindo a máscara
    var[:] = variavel_existente.filled(missing_value)

    Y.units = 'degrees_north'
    X.units = 'degrees_east'
    T.units = 'months since 01-Jan'
    var.long_name = variavel_long_name
    var.units = variavel_units
    var.history = variavel_history
    var.missing_value = missing_value

    # Fechando o arquivo NetCDF
    ncid.close()
    print(f'Arquivo {filename} criado com sucesso!')

### airdens.cdf

filename = 'airdens.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'airdens' <br>
variavel_existente = airdens <br>
var_long_name = 'sea level air density' <br>
variavel_units = 'kilogram/meter3' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [29]:
filename = 'airdens.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'airdens'
variavel_existente = airdens
variavel_long_name = 'sea level air density'
variavel_units = 'kilogram/meter3'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [31]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo airdens.nc criado com sucesso!


### emp.cdf

filename = 'emp.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'emp' <br>
variavel_existente = emp <br>
var_long_name = 'constrained evaporation minus precipitation' <br>
variavel_units = 'mm/(3hour)' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [48]:
filename = 'emp.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'emp'
variavel_existente = emp
variavel_long_name = 'constrained evaporation minus precipitation'
variavel_units = 'mm/(3hour)'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [50]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo emp.nc criado com sucesso!


### netheat.cdf

filename = 'netheat.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'netheat' <br>
variavel_existente = clim_netheat <br>
variavel_long_name = 'constrained new outgoing heat flux' <br>
variavel_units = 'W/m2' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [91]:
filename = 'netheat.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'netheat'
variavel_existente = netheat
variavel_long_name = 'constrained new outgoing heat flux'
variavel_units = 'W/m2'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [93]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo netheat.nc criado com sucesso!


### precip.cdf

filename = 'precip.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'precip' <br>
variavel_existente = clim_tp <br>
variavel_long_name = 'precipitation rate' <br>
variavel_units = 'mm/(3hour)' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [55]:
filename = 'precip.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'precip'
variavel_existente = precip
variavel_long_name = 'precipitation rate'
variavel_units = 'mm/hour'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [56]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo precip.nc criado com sucesso!


### qsea.cdf

filename = 'qsea.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'qsea' <br>
variavel_existente = qsea <br>
variavel_long_name = 'sea level specific humidity' <br>
variavel_units = 'g/kg' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [25]:
filename = 'qsea.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'qsea'
variavel_existente = qsea
variavel_long_name = 'sea level specific humidity'
variavel_units = 'g/kg'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [27]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo qsea.nc criado com sucesso!


### sat.cdf

filename = 'sat.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'sat' <br>
variavel_existente = sat <br>
variavel_long_name = 'sea level air temperature' <br>
variavel_units = 'Celsius_scale' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [33]:
filename = 'sat.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'sat'
variavel_existente = sat
variavel_long_name = 'sea level air temperature'
variavel_units = 'Celsius_scale'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [35]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo sat.nc criado com sucesso!


### shortrad.cdf

filename = 'shortrad.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'shortrad' <br>
variavel_existente = shortrad <br>
variavel_long_name = 'net short wave radiation' <br>
variavel_units = 'W/m2' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [61]:
filename = 'shortrad.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'shortrad'
variavel_existente = shortrad
variavel_long_name = 'net short wave radiation'
variavel_units = 'W/m2'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [62]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo shortrad.nc criado com sucesso!


### taux.cdf

filename = 'taux.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'taux' <br>
variavel_existente = taux <br>
variavel_long_name = 'zonal wind stress' <br>
variavel_units = 'newton/meter2' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [63]:
filename = 'taux.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'taux'
variavel_existente = taux
variavel_long_name = 'zonal wind stress'
variavel_units = 'newton/meter2'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [64]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo taux5.nc criado com sucesso!


### tauy.cdf

filename = 'tauy.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'tauy' <br>
variavel_existente = taux <br>
variavel_long_name = 'meridional wind stress' <br>
variavel_units = 'newton/meter2' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [65]:
filename = 'tauy.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
var_str = 'tauy'
time = time_nc
variavel_existente = tauy
variavel_long_name = 'meridional wind stress'
variavel_units = 'newton/meter2'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [66]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo tauy.nc criado com sucesso!


### u3.cdf

filename = 'u3.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'u3' <br>
variavel_existente = u3 <br>
variavel_long_name = 'zonal wind' <br>
variavel_units = 'meter/second' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [67]:
filename = 'u3.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'u3'
variavel_existente = u3
variavel_long_name = 'zonal wind'
variavel_units = 'meter/second'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [68]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo u3.nc criado com sucesso!


### v3.cdf

filename = 'v3.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'v3' <br>
variavel_existente = v3 <br>
variavel_long_name = 'meridional wind' <br>
variavel_units = 'meter/second' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [69]:
filename = 'v3.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'v3'
variavel_existente = v3
variavel_long_name = 'meridional wind'
variavel_units = 'meter/second'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [70]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo v3.nc criado com sucesso!


### w3.cdf

filename = 'w3.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'w3' <br>
variavel_existente = w3 <br>
variavel_long_name = 'wind speed' <br>
variavel_units = 'meter/second' <br>
variavel_history = 'surface marine climatology 2010-2023' <br>
missing_value = -10000000000 <br>

In [71]:
filename = 'w3.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'w3'
variavel_existente = w3
variavel_long_name = 'wind speed'
variavel_units = 'meter/second'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [72]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo w3.nc criado com sucesso!


### sst.cdf

filename = 'sst.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'sst' <br>
variavel_existente = sst <br>
variavel_long_name = 'sea surface temperature' <br>
variavel_units = 'Celsius_scale' <br>
variavel_history = 'surface marine climatology 2010-2022' <br>
missing_value = -10000000000 <br>

In [73]:
filename = 'sst.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'sst'
variavel_existente = sst
variavel_long_name = 'sea surface temperature'
variavel_units = 'Celsius_scale'
variavel_history = 'surface marine climatology 2000-2023'
missing_value = -1.0e10

In [74]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo sst.nc criado com sucesso!


### sss.cdf

filename = 'sss.nc' <br>
timedim = len(time_nc) <br>
latdim = len(lat) <br>
londim = len(lon) <br>
var_str = 'salinity' <br>
variavel_existente = sss <br>
variavel_long_name = 'salinity' <br>
variavel_units = 'PSU' <br>
variavel_history = 'surface marine climatology 2010-2022' <br>
missing_value = -10000000000 <br>

In [75]:
filename = 'sss.nc'
timedim = len(time_nc)
latdim = len(lat)
londim = len(lon)
time = time_nc
var_str = 'salinity'
variavel_existente = sss
variavel_long_name = 'salinity'
variavel_units = 'PSU'
variavel_history = 'surface marine climatology 1955-2017'
missing_value = -1.0e10

In [76]:
get_netcdf(filename, timedim, latdim, londim, lat, lon, time, var_str, variavel_existente, variavel_long_name, variavel_units, variavel_history, missing_value)

Arquivo sss.nc criado com sucesso!
